<a href="https://colab.research.google.com/github/eldercamposds/AnaliseDeSentimento/blob/LSTM/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np

In [2]:
Tweets = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NLP/7.Analise de sentimentos/Tweets.csv')

In [3]:
Tweets.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [4]:
Tweets.tail(3)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
14637,569587242672398336,neutral,1.0000,NaN,NaN,American,NaN,sanyabun,NaN,0,@AmericanAir Please bring American Airlines to...,NaN,2015-02-22 11:59:15 -0800,"Nigeria,lagos",NaN
14638,569587188687634433,negative,1.0000,Customer Service Issue,0.6659,American,NaN,SraJackson,NaN,0,"@AmericanAir you have my money, you change my ...",NaN,2015-02-22 11:59:02 -0800,New Jersey,Eastern Time (US & Canada)
14639,569587140490866689,neutral,0.6771,NaN,0.0000,American,NaN,daviddtwu,NaN,0,@AmericanAir we have 8 ppl so we need 2 know h...,NaN,2015-02-22 11:58:51 -0800,"dallas, TX",NaN


In [5]:
Tweets.groupby(["airline_sentiment"]).size()

,0
airline_sentiment,
negative,9178
neutral,3099
positive,2363


#Tratamento

In [6]:
Tweets = Tweets[Tweets["airline_sentiment_confidence"] > 0.8]

In [7]:
Tweets.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
3,570301031407624196,negative,1.0,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)
5,570300767074181121,negative,1.0,Can't Tell,0.6842,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica seriously would pay $30 a fligh...,NaN,2015-02-24 11:14:33 -0800,NaN,Pacific Time (US & Canada)
9,570295459631263746,positive,1.0,NaN,NaN,Virgin America,NaN,YupitsTate,NaN,0,"@VirginAmerica it was amazing, and arrived an ...",NaN,2015-02-24 10:53:27 -0800,Los Angeles,Eastern Time (US & Canada)


In [8]:
max_words=100

In [9]:
token = Tokenizer(num_words=max_words)
token.fit_on_texts(Tweets["text"].values) #criando o modelo

In [10]:
X = token.texts_to_sequences(Tweets["text"].values)#gerando tokens
X = pad_sequences(X, padding="post", maxlen=max_words) #ccompletar tokens com 0 a direita

In [11]:
print(X)

[[97 62  0 ...  0  0  0]
 [97 99  1 ...  0  0  0]
 [97  9 99 ...  0  0  0]
 ...
 [13 98 93 ...  0  0  0]
 [13 89  1 ...  0  0  0]
 [13  6 23 ...  0  0  0]]


In [12]:
X.shape

(10459, 100)

In [13]:
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(Tweets["airline_sentiment"])
print(y)

[1 0 0 ... 0 1 0]


In [14]:
y  = to_categorical(y)
print(y)

[[0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]


In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
X_test

array([[16, 87, 45, ...,  0,  0,  0],
       [16, 91,  4, ...,  0,  0,  0],
       [12,  3,  2, ...,  0,  0,  0],
       ...,
       [12, 15,  9, ...,  0,  0,  0],
       [97,  3,  4, ...,  0,  0,  0],
       [13,  7, 64, ...,  0,  0,  0]], dtype=int32)

#Modelos

In [16]:
modelo = Sequential()
modelo.add(Embedding(input_dim = len(token.word_index), output_dim=128))
modelo.add(SpatialDropout1D(0.2))#auemnta a independencia entre as caracteristicas
modelo.add(LSTM(units=196, dropout=0.2, recurrent_dropout=0, activation="tanh", recurrent_activation="sigmoid", use_bias=True))
modelo.add(Dense(units=3, activation="softmax"))

In [17]:
modelo.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [18]:
print(modelo.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ spatial_dropout1d (SpatialDropout1D) │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


#Treino

In [19]:
modelo.fit(X_train, y_train, epochs=10, batch_size=30, verbose=True, validation_data=(X_test, y_test))

Epoch 1/10
245/245 ━━━━━━━━━━━━━━━━━━━━ 74s 290ms/step - accuracy: 0.7025 - loss: 0.8271 - val_accuracy: 0.7020 - val_loss: 0.8191
Epoch 2/10
245/245 ━━━━━━━━━━━━━━━━━━━━ 76s 266ms/step - accuracy: 0.7048 - loss: 0.8175 - val_accuracy: 0.7020 - val_loss: 0.8157
Epoch 3/10
245/245 ━━━━━━━━━━━━━━━━━━━━ 65s 264ms/step - accuracy: 0.7107 - loss: 0.8055 - val_accuracy: 0.7020 - val_loss: 0.8169
Epoch 4/10
245/245 ━━━━━━━━━━━━━━━━━━━━ 82s 265ms/step - accuracy: 0.7200 - loss: 0.7905 - val_accuracy: 0.7020 - val_loss: 0.8206
Epoch 5/10
245/245 ━━━━━━━━━━━━━━━━━━━━ 89s 293ms/step - accuracy: 0.7076 - loss: 0.8088 - val_accuracy: 0.7020 - val_loss: 0.8160
Epoch 6/10
245/245 ━━━━━━━━━━━━━━━━━━━━ 76s 268ms/step - accuracy: 0.7192 - loss: 0.7908 - val_accuracy: 0.7020 - val_loss: 0.8159
Epoch 7/10
245/245 ━━━━━━━━━━━━━━━━━━━━ 94s 317ms/step - accuracy: 0.7110 - loss: 0.8035 - val_accuracy: 0.7020 - val_loss: 0.8196
Epoch 8/10
245/245 ━━━━━━━━━━━━━━━━━━━━ 65s 267ms/step - accuracy: 0.7157 - loss: 0

In [21]:
loss, accuracy = modelo.evaluate(X_test, y_test)
print("Loss: ", loss)
print("Acurácia: ", accuracy)

99/99 ━━━━━━━━━━━━━━━━━━━━ 9s 89ms/step - accuracy: 0.6982 - loss: 0.8267
Loss:  0.8200196027755737
Acurácia:  0.7020395398139954


In [20]:
prev =  modelo.predict(X_test)
print(prev)

99/99 ━━━━━━━━━━━━━━━━━━━━ 9s 90ms/step
[[0.7435236  0.12840283 0.12807356]
 [0.7435236  0.12840283 0.12807354]
 [0.7435236  0.12840283 0.12807356]
 ...
 [0.7435236  0.12840283 0.12807356]
 [0.7435236  0.12840283 0.12807356]
 [0.7435236  0.12840283 0.12807356]]
